In [1]:
import glob

import xarray as xr
import xclim as xc

/g/data/xv83/dbi599/miniconda3/envs/xclim/lib/python3.13/site-packages/xclim/core/units.py:19: UserWarning: Import(s) unavailable to set up matplotlib support...skipping this portion of the setup.
  import cf_xarray.units


In [19]:
print(xc.__version__)

0.56.0


### Read and prepare input data

In [2]:
def convert_units(da, target_units):
    """Convert units.

    Parameters
    ----------
    da : xarray DataArray
        Input array containing a units attribute
    target_units : str
        Units to convert to

    Returns
    -------
    da : xarray DataArray
       Array with converted units
    """

    xclim_unit_check = {
        'degrees_Celsius': 'degC',
        'deg_k': 'degK',
    }

    if da.attrs["units"] in xclim_unit_check:
        da.attrs["units"] = xclim_unit_check[da.units]

    try:
        with xr.set_options(keep_attrs=True):
            da = xc.units.convert_units_to(da, target_units)
    except Exception as e:
        if (da.attrs['units'] == 'kg m-2 s-1') and (target_units in ['mm d-1', 'mm day-1']):
            da = da * 86400
            da.attrs["units"] = target_units
        elif (da.attrs['units'] == 'MJ m^-2') and target_units == 'W m-2':
            da = da * (1e6 / 86400)
            da.attrs["units"] = target_units
        else:
            raise e
    
    if target_units == 'degC':
        da.attrs['units'] = 'degC'

    return da

In [3]:
def select_point(ds):
    """Select grid point nearest to Kinglake, Victoria"""
    
    ds = ds.sel({'lat': -37.53, 'lon': 145.34}, method='nearest')
    vars_to_drop = [
        'sigma',
        'level_height',
        'height',
        'crs',
    ]
    for variable in vars_to_drop:
        try:
            ds = ds.drop_vars(variable)
        except ValueError:
            pass
    
    attrs_to_del = ['cell_methods', 'grid_mapping']
    variables = list(ds.keys())
    for variable in variables:
        for attr in attrs_to_del:
            try:
                del ds[variable].attrs[attr]
            except KeyError:
                pass
        
    return ds

In [4]:
pr_files = sorted(glob.glob('/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/pr/latest/*.nc'))

In [5]:
ds_pr_daily = xr.open_mfdataset(pr_files, preprocess=select_point)
ds_pr_daily = ds_pr_daily.sel(time=slice('1980-01-01', '2023-12-31'))
da_pr_daily = convert_units(ds_pr_daily['pr'], 'mm/day')
da_pr_daily = da_pr_daily.compute()

In [6]:
da_pr_annual = da_pr_daily.groupby('time.year').sum('time', keep_attrs=True)
da_pr_annual_mean = da_pr_annual.mean('year', keep_attrs=True)
da_pr_annual_mean.attrs['units'] = 'mm/year'
da_pr_annual_mean = da_pr_annual_mean.compute()

In [7]:
tasmax_files = sorted(glob.glob('/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/tasmax/latest/*.nc'))

In [8]:
ds_tasmax_daily = xr.open_mfdataset(tasmax_files, preprocess=select_point)
ds_tasmax_daily = ds_tasmax_daily.sel(time=slice('1980-01-01', '2023-12-31'))
da_tasmax_daily = convert_units(ds_tasmax_daily['tasmax'], 'degC')
da_tasmax_daily = da_tasmax_daily.compute()

In [15]:
hurs_files = sorted(glob.glob('/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/hurs/latest/*.nc'))

In [18]:
ds_hurs_daily = xr.open_mfdataset(hurs_files, preprocess=select_point)
da_hurs_daily = ds_hurs_daily['hurs'].sel(time=slice('1980-01-01', '2023-12-31'))
da_hurs_daily = da_hurs_daily.compute()

In [21]:
sfcWind_files = sorted(glob.glob('/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/sfcWind/latest/*.nc'))

In [23]:
ds_sfcWind_daily = xr.open_mfdataset(sfcWind_files, preprocess=select_point)
da_sfcWind_daily = ds_sfcWind_daily['sfcWind'].sel(time=slice('1980-01-01', '2023-12-31'))
da_sfcWind_daily = da_sfcWind_daily.compute()

### Calculate the FFDI

In [25]:
da_kbdi_daily = xc.indices.keetch_byram_drought_index(da_pr_daily, da_tasmax_daily, da_pr_annual_mean)

In [26]:
da_df_daily = xc.indices.griffiths_drought_factor(da_pr_daily, da_kbdi_daily)

In [27]:
da_ffdi_daily = xc.indices.mcarthur_forest_fire_danger_index(da_df_daily, da_tasmax_daily, da_hurs_daily, da_sfcWind_daily)

In [31]:
da_ffdi_daily.sel(time=slice('2008-01-01', '2009-12-31')).plot()

ImportError: The matplotlib package is required for plotting but could not be imported. Please install it with your package manager (e.g. conda or pip).

In [ ]:
#def calculate_ffdi(pra, tasmaxa, pr_annual, hursmina, sfcWindmaxa):
#    KBDI = keetch_byram_drought_index(pra, tasmaxa, pr_annual)
#    DF = griffiths_drought_factor(pra, KBDI)
#    FFDI = mcarthur_forest_fire_danger_index(DF, tasmaxa, hursmina, sfcWindmaxa)
#    return FFDI.to_dataset(name='FFDI')